In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 31 12:19:01 2023

@author: 15391
"""
import os
import re
from lxml import etree
import requests
import pandas as pd

from bs4 import BeautifulSoup

class my_spider():
    
    def __init__(self,url,save_path):
        self.url = url
        self.save_path=save_path
        self.title=[]
        self.rating_num=[]
        self.inq=[]
        self.href=[]
        self.im_url=[]
    def request(self,url):
        self.headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"}
        html = requests.get(url,headers=self.headers)
        return html
    def parse_bs4(self):
        for i in range(10):
            url=self.url+'?start='+str(i*25)+'&filter='
            print(f"正在爬取：{url}")
            html=self.request(url)
            self.soup = BeautifulSoup(html.text, 'html.parser')

            movie_items= self.soup.find_all('div', class_='info')
            for item in movie_items:
                self.title.append(item.find('span',class_='title').get_text())
                self.rating_num.append(item.find('span',class_='rating_num').get_text())
                if item.find('span',class_='inq')!=None:
                    self.inq.append(item.find('span',class_='inq').get_text())
                else:
                    self.inq.append("此电影无短评")
                self.href.append(item.find('div',class_='hd').a.get("href"))
        
            
            
            html_list=self.soup.find_all('div', class_='item')
            for html in html_list:
                self.im_url.append(html.find_all('img')[0]['src'])

    def parse_lxml(self):
        for i in range(10):
            url=self.url+'?start='+str(i*25)+'&filter='
            print(f"正在爬取：{url}")
            html=self.request(url)
            dencoding=requests.get(url,headers=self.headers).encoding
            self.data=etree.HTML(html.content,parser=etree.HTMLParser(encoding=dencoding))
            self.href+=self.data.xpath("//div[@class='info']/div[@class='hd']/a/@href")
            self.title+=self.data.xpath("//div[@class='info']//span[@class='title'][1]//text()")
            self.rating_num+=self.data.xpath("//div[@class='info']//div[@class='star']//span[@class='rating_num']//text()")
            self.inq+=self.data.xpath("//div[@class='info']//p[@class='quote']//span[@class='inq']//text()")
    
    def save(self):
        self.df = pd.DataFrame()  # 初始化一个DataFrame对象
        self.df['电影名称'] = self.title 
        self.df['评分'] = self.rating_num
        self.df['评语'] = self.inq
        self.df['链接'] = self.href

        sess = requests.session()
        for (url,name) in zip(self.im_url,self.title):
            res=sess.get(url)
            with open('./picture/'+name+'.jpg', 'wb') as f:
                f.write(res.content)
        
        self.df.to_excel(self.save_path)
model=my_spider(url='https://movie.douban.com/top250',save_path='top250.xlsx')
model.parse_bs4()
model.save()

正在爬取：https://movie.douban.com/top250?start=0&filter=
